# TODO
- More plots and EDA
- Finish volatility (see below) - applying after forecast
- cross-validation and model evaluation
- output for visualization

In [1]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
import numpy as np
import gcsfs
from sklearn.metrics import mean_squared_error

## Load Data

In [2]:
fs = gcsfs.GCSFileSystem(project='cse6242-project-476314', token = 'google_default')
with fs.open('cse6242-project-data-bucket/Zip_zori_uc_sfrcondomfr_sm_sa_month.csv') as f:
    zori_wide = pd.read_csv(f)

zori_wide

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1501.705581,...,1902.035623,1902.589323,1894.295923,1891.247836,1889.899732,1889.783368,1886.582030,1887.378619,1888.256767,1891.123162
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1252.988450,...,1834.055010,1834.801216,1836.145502,1835.505043,1831.819858,1833.617479,1829.740778,1831.288573,1825.781679,1829.945544
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1127.656461,...,1625.302516,1624.803898,1632.635241,1632.612081,1630.639582,1615.873819,1614.456853,1619.566427,1630.300973,1634.061782
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1426.444928,1435.425031,1443.855231,1446.723283,1441.421688,1439.302928,1441.642569,1448.088589,1456.518933,1463.357895
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,...,3150.113998,3186.884070,3216.427094,3224.476398,3223.466612,3234.890106,3255.649669,3275.779831,3277.512495,3300.138507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,418163,30158,89158,zip,NV,NV,Las Vegas,"Las Vegas-Henderson-Paradise, NV",Clark County,NaN,...,NaN,NaN,NaN,NaN,NaN,3066.735205,3141.404502,3220.651060,NaN,3383.333333
7849,70410,30172,29333,zip,SC,SC,Spartanburg,"Spartanburg, SC",Spartanburg County,NaN,...,NaN,NaN,NaN,NaN,1496.722526,1492.599968,1467.567991,1471.938011,1447.380379,1441.444444
7850,61618,30490,10004,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,...,4953.861191,4992.794726,5020.881625,4966.573186,4993.118270,4985.137479,5019.837773,5064.354786,5128.502857,5207.319444
7851,91179,30490,76005,zip,TX,TX,Arlington,"Dallas-Fort Worth-Arlington, TX",Tarrant County,NaN,...,2266.033986,2269.037047,2240.741193,2237.569443,2215.046263,2215.363392,2210.884130,2224.301864,2214.883471,2216.444444


In [3]:
# get into long format
value_vars = [col for col in zori_wide.columns if col[:4].isdigit()]

zori = zori_wide.melt(
    id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'],
    value_vars = value_vars,
    var_name='Date',
    value_name='Rent'
)

# format date
zori['Date'] = pd.to_datetime(zori['Date'])

# rename RegionID to ZIP
zori = zori.rename(columns={'RegionID':'ZIP'})

# sort time series
zori = zori.sort_values(['ZIP', 'Date'])

zori

,ZIP,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Date,Rent
3692,58197,3911,1002,zip,MA,MA,Amherst,"Springfield, MA",Hampshire County,2015-01-31,NaN
11545,58197,3911,1002,zip,MA,MA,Amherst,"Springfield, MA",Hampshire County,2015-02-28,NaN
19398,58197,3911,1002,zip,MA,MA,Amherst,"Springfield, MA",Hampshire County,2015-03-31,NaN
27251,58197,3911,1002,zip,MA,MA,Amherst,"Springfield, MA",Hampshire County,2015-04-30,NaN
35104,58197,3911,1002,zip,MA,MA,Amherst,"Springfield, MA",Hampshire County,2015-05-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...
979408,845914,6361,85288,zip,AZ,AZ,Tempe,"Phoenix-Mesa-Chandler, AZ",Maricopa County,2025-05-31,2000.823720
987261,845914,6361,85288,zip,AZ,AZ,Tempe,"Phoenix-Mesa-Chandler, AZ",Maricopa County,2025-06-30,2003.705718
995114,845914,6361,85288,zip,AZ,AZ,Tempe,"Phoenix-Mesa-Chandler, AZ",Maricopa County,2025-07-31,2020.338279
1002967,845914,6361,85288,zip,AZ,AZ,Tempe,"Phoenix-Mesa-Chandler, AZ",Maricopa County,2025-08-31,2046.337532


## Data Inspection

In [4]:
zori.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1013037 entries, 3692 to 1010820
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   ZIP         1013037 non-null  int64         
 1   SizeRank    1013037 non-null  int64         
 2   RegionName  1013037 non-null  int64         
 3   RegionType  1013037 non-null  object        
 4   StateName   1013037 non-null  object        
 5   State       1013037 non-null  object        
 6   City        1005555 non-null  object        
 7   Metro       1008006 non-null  object        
 8   CountyName  1013037 non-null  object        
 9   Date        1013037 non-null  datetime64[ns]
 10  Rent        396233 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 92.7+ MB


In [5]:
zori.describe()

,ZIP,SizeRank,RegionName,Date,Rent
count,1.013037e+06,1.013037e+06,1.013037e+06,1013037,396233.000000
mean,8.469800e+04,4.770870e+03,5.263820e+04,2020-05-30 20:39:04.186046720,1814.321472
min,5.819700e+04,1.000000e+00,1.002000e+03,2015-01-31 00:00:00,528.828904
25%,7.036500e+04,2.037000e+03,2.915000e+04,2017-09-30 00:00:00,1301.376023
50%,7.948800e+04,4.178000e+03,4.842300e+04,2020-05-31 00:00:00,1649.037387
75%,9.326100e+04,6.735000e+03,7.993200e+04,2023-01-31 00:00:00,2128.800125
max,8.459140e+05,3.049000e+04,9.980100e+04,2025-09-30 00:00:00,99629.555556
std,4.216436e+04,3.566825e+03,2.993071e+04,NaN,1108.741032


In [6]:
# missing values
zori.isna().sum()

ZIP                0
SizeRank           0
RegionName         0
RegionType         0
StateName          0
State              0
City            7482
Metro           5031
CountyName         0
Date               0
Rent          616804
dtype: int64

In [7]:
# spacing
zori['gap_days']  = zori.groupby('ZIP')['Date'].diff().dt.days
anomalies = zori[zori['gap_days'] > 31]
anomalies.head()

,ZIP,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Date,Rent,gap_days


In [8]:
# Handle missingness for Rent - forecasts will either skip over missing data (ETS) or distort mean (averages/std for volatility)
# TODO: really handle it this way?
# TODO: check for gaps graphically
zori['Rent'] = (
    zori.groupby('ZIP')['Rent']
        .transform(lambda s: s.interpolate(method='linear', limit_direction='both'))
)
# formatting Date to prevent future warning
zori['Date'] = zori['Date'].dt.to_period('M').dt.to_timestamp('M') 
zori.isna().sum()

ZIP              0
SizeRank         0
RegionName       0
RegionType       0
StateName        0
State            0
City          7482
Metro         5031
CountyName       0
Date             0
Rent             0
gap_days      7853
dtype: int64

In [9]:
zori['gap_days']  = zori.groupby('ZIP')['Date'].diff().dt.days
anomalies = zori[zori['gap_days'] > 31]
anomalies.head()

,ZIP,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Date,Rent,gap_days


## Forecast

In [10]:
def forecast_zip(group):
    '''
    This runs for each ZIP
    '''
    group = group.sort_values('Date').copy()

    train = group.set_index('Date')['Rent'].asfreq('ME')
    # skip short series
    #if len(train) < 24:
    #    return pd.DataFrame()  

    # Exponential Smoothing
    #model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
    model = ExponentialSmoothing(train, trend='add', seasonal=None)
    fit = model.fit()
    forecast = fit.forecast(12)

    # Linear Baseline
    #group['t'] = np.arange(len(group))
    y = train.values
    t = np.arange(len(train)).reshape(-1,1)
    lr = LinearRegression().fit(t, y)
    #lr = LinearRegression().fit(group[['t']], group['Rent'])
    #future_t = np.arange(len(group), len(group)+12).reshape(-1,1)
    future_t = np.arange(len(train), len(train)+12).reshape(-1, 1)
    forecast_lr = lr.predict(future_t)

    # Combine the forecasts into a dataframe
    future_dates = pd.date_range(group['Date'].max() + pd.offsets.MonthBegin(), periods=12, freq='MS')

    df_fore = pd.DataFrame({
        'ZIP': group['ZIP'].iloc[0],
        'Date': future_dates,
        'Forecast_ExpSmooth': forecast.values,
        'Forecast_Linear': forecast_lr
    })

    return df_fore

In [11]:
# apply the forecasts (baseline linear and exponential smoothing) to all zipcodes
# TODO - temporary filter for testing
zori = zori[zori['State'] == 'GA']
all_forecasts = (
    zori.groupby('ZIP', group_keys=False)
    .apply(forecast_zip)
    .reset_index(drop=True)
)

# TODO: after forecasting apply volatility
#group['pct_change'] = group['Rent'].pct_change()
#group['volatility'] = group['pct_change'].rolling(12).std()

/var/folders/v6/rt7shrqx5sldb75bkwymnhlw0000gp/T/ipykernel_44257/824115300.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(forecast_zip)


In [12]:
# extract our target forecasts - 3, 6, 9, 12 month
horizons = [3, 6, 9, 12]
summary = []

for zip_code, df in all_forecasts.groupby('ZIP'):
    for h in horizons:
        if len(df) >= h:
            summary.append({
                'ZIP': zip_code,
                'Horizon_Months': h,
                'ExpSmooth': df['Forecast_ExpSmooth'].iloc[h-1],
                'Linear': df['Forecast_Linear'].iloc[h-1]
            })

zip_forecast_summary = pd.DataFrame(summary)

In [13]:
# TODO: save somewhere our where does it go? send to gcp?
#zip_forecast_summary.to_csv('ZIP_forecasts.csv', index=False)

## Evaluatation

In [14]:
all_forecasts['Date'] = pd.to_datetime(all_forecasts['Date'])
zori_merge = (zori.merge(all_forecasts, on=['ZIP', 'Date'], how='left'))
zori_merge

,ZIP,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Date,Rent,gap_days,Forecast_ExpSmooth,Forecast_Linear
0,70791,282,30004,zip,GA,GA,Milton,"Atlanta-Sandy Springs-Alpharetta, GA",Fulton County,2015-01-31,1237.885999,NaN,NaN,NaN
1,70791,282,30004,zip,GA,GA,Milton,"Atlanta-Sandy Springs-Alpharetta, GA",Fulton County,2015-02-28,1233.552236,28.0,NaN,NaN
2,70791,282,30004,zip,GA,GA,Milton,"Atlanta-Sandy Springs-Alpharetta, GA",Fulton County,2015-03-31,1240.469951,31.0,NaN,NaN
3,70791,282,30004,zip,GA,GA,Milton,"Atlanta-Sandy Springs-Alpharetta, GA",Fulton County,2015-04-30,1247.236144,30.0,NaN,NaN
4,70791,282,30004,zip,GA,GA,Milton,"Atlanta-Sandy Springs-Alpharetta, GA",Fulton County,2015-05-31,1258.633373,31.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31342,71721,2031,31909,zip,GA,GA,Columbus,"Columbus, GA-AL",Muscogee County,2025-05-31,1536.867715,31.0,NaN,NaN
31343,71721,2031,31909,zip,GA,GA,Columbus,"Columbus, GA-AL",Muscogee County,2025-06-30,1534.918098,30.0,NaN,NaN
31344,71721,2031,31909,zip,GA,GA,Columbus,"Columbus, GA-AL",Muscogee County,2025-07-31,1540.749010,31.0,NaN,NaN
31345,71721,2031,31909,zip,GA,GA,Columbus,"Columbus, GA-AL",Muscogee County,2025-08-31,1535.780142,31.0,NaN,NaN


In [15]:
def rmse(series, forecast):
    return mean_squared_error(series[-len(forecast):], forecast)

In [16]:
# TODO: Use cross-validation and do some testing